## Problem statement

We want to predict the grades of students given features such as gender, parental education, race and test prep course. Additionally, we will rank these features in terms of their contribution to the final prediction.

The dataset has grades for 3 different categories - math, reading , writing. We will combine these three scores and map these to a grading scheme (usually followed in most institutions). The problem will then be reduced to a classification problem.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data=pd.read_csv('StudentsPerformance.csv')

In [3]:
data['Percentage'] = round((data['reading score'] + data['writing score'] + data['math score']) / 3, 2)

In [4]:
def Grade(percentage):
    if percentage >= 93: return "A"
    if percentage > 81 : return "B"
    if percentage > 71 : return "C"
    if percentage > 61 : return "D"
    if percentage > 51 : return "E"
    else: return "F"

data["Grade"] = data['Percentage'].apply(lambda x: Grade(x))

In [6]:
data.tail()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,Percentage,Grade
995,female,group E,master's degree,standard,completed,88,99,95,94.00,A
996,male,group C,high school,free/reduced,none,62,55,55,57.33,E
997,female,group C,high school,free/reduced,completed,59,71,65,65.00,D
998,female,group D,some college,standard,completed,68,78,77,74.33,C
999,female,group D,some college,free/reduced,none,77,86,86,83.00,B


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gender                       1000 non-null   object 
 1   race/ethnicity               1000 non-null   object 
 2   parental level of education  1000 non-null   object 
 3   lunch                        1000 non-null   object 
 4   test preparation course      1000 non-null   object 
 5   math score                   1000 non-null   int64  
 6   reading score                1000 non-null   int64  
 7   writing score                1000 non-null   int64  
 8   Percentage                   1000 non-null   float64
 9   Grade                        1000 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 78.3+ KB
